# ICAPS 2024 Tutorial: Finding multiple plans - K*

K* planner can be installed via pip.

In [1]:
from IPython.display import clear_output

!pip install kstar-planner==1.4.2
clear_output(wait=False)

## Domain and Problem Files
We use the iconic problem number one of the gripper domain, where the goal is to move two
balls from room A to room B with a gripper that has two arms.

<br>
<img src="https://github.com/mp-tutorial/mp-tutorial.github.io/blob/main/notebooks/gripper-two-balls.png?raw=true" width=600>
<br>

We need to download the domain and problem files described in PDDL for the planning task we aim to solve. Following this, we read and use these files to create a PDDL problem, setting the quality metric to minimize the plan length.

In [2]:
!wget https://raw.githubusercontent.com/mp-tutorial/mp-tutorial.github.io/main/notebooks/gripper-domain.pddl
!wget https://raw.githubusercontent.com/mp-tutorial/mp-tutorial.github.io/main/notebooks/gripper-prob-two-balls.pddl

def count_plans_by_cost(plans):
    res = {}
    for plan in plans:
        c = plan["cost"]
        if c not in res:
            res[c] = 0
        res[c] += 1
    return res

from kstar_planner import planners
from pathlib import Path

domain_file = Path("gripper-domain.pddl")
problem_file = Path("gripper-prob-two-balls.pddl")

clear_output(wait=False)

## Top-k planning

In [3]:
import json
# Top-k planning, OK* with LMCut heuristic by default
plans = planners.plan_topk(domain_file=domain_file, problem_file=problem_file, number_of_plans_bound=1000, timeout=30)
print(count_plans_by_cost(plans["plans"]))

{5: 8, 6: 8, 7: 144, 8: 144, 9: 696}


### Planner output

In [4]:
print(json.dumps(plans, indent=4))


{
    "planner_output": "INFO     Running translator.\nINFO     translator stdin: None\nINFO     translator time limit: None\nINFO     translator memory limit: None\nINFO     translator command line string: /Users/michaelkatz/software/Challenges/llms-planning-data-gen/.venv/bin/python /Users/michaelkatz/software/Challenges/llms-planning-data-gen/.venv/lib/python3.11/site-packages/kstar_planner/builds/release/bin/translate/translate.py /Users/michaelkatz/software/web/mp-tutorial.github.io/notebooks/gripper-domain.pddl /Users/michaelkatz/software/web/mp-tutorial.github.io/notebooks/gripper-prob-two-balls.pddl --sas-file output.sas\nParsing...\nParsing: [0.000s CPU, 0.001s wall-clock]\nNormalizing task... [0.000s CPU, 0.000s wall-clock]\nInstantiating...\nGenerating Datalog program... [0.000s CPU, 0.000s wall-clock]\nNormalizing Datalog program...\nNormalizing Datalog program: [0.000s CPU, 0.001s wall-clock]\nPreparing model... [0.000s CPU, 0.000s wall-clock]\nGenerated 24 rules.\nComputi

## Top-quality planning

In [5]:
# Top-quality planning, OK* with LMCut heuristic by default
plans = planners.plan_topq(domain_file=domain_file, problem_file=problem_file, quality_bound=1.4, number_of_plans_bound=1000, timeout=30)
print(count_plans_by_cost(plans["plans"]))

{5: 8, 6: 8, 7: 144}


## Unordered top-quality planning 

In [6]:
heuristic = "ipdb(transform=undo_to_origin())"

# Unordered top-quality planning, ORK* with iPDB heuristic
plans = planners.plan_unordered_topq(domain_file=domain_file, problem_file=problem_file, quality_bound=1.4, number_of_plans_bound=1000, timeout=30, search_heuristic=heuristic)
print(count_plans_by_cost(plans["plans"]))

{5: 2, 6: 2, 7: 20}
